In [32]:
# Laurent LEQUIEVRE
# Research Engineer, CNRS (France)
# Institut Pascal UMR6602
# laurent.lequievre@uca.fr

In [63]:
# torch.nn contains loss functions

import torch
import torch.nn as nn

In [166]:
my_device = torch.device("cuda") # can be also "cpu"
my_dtype = torch.float

# Create random Tensors

# This returns a tensor of size 3 × 5
# filled with values from standard normal distribution, that is mean is 0 and variance is 1.
# Setting requires_grad=True indicates that we want to compute gradients during the backward pass.
input = torch.randn(3, 5, device=my_device, dtype=my_dtype, requires_grad=True) 

target = torch.randn(3, 5, device=my_device, dtype=my_dtype)

print('input: ', input)
print('target: ', target)

input:  tensor([[-0.9320,  1.6002,  0.3595,  1.8899, -0.1184],
        [-0.8379,  0.0553,  0.8553, -0.3518,  0.8582],
        [-0.7784, -0.9570, -1.0201,  1.6142,  2.0713]], device='cuda:0',
       requires_grad=True)
target:  tensor([[-2.8785e-01,  3.5644e-04,  1.3720e-01,  1.8768e+00,  4.2386e-01],
        [ 9.7907e-02,  1.8768e+00, -3.8424e-01,  1.6855e+00,  1.0096e+00],
        [ 6.5938e-01, -1.1361e-01, -2.6892e-01, -1.4951e+00,  6.3044e-01]],
       device='cuda:0')


In [167]:
# Get size of tensors input and target
print("input size={}, target size={}".format(input.shape,target.shape))

input size=torch.Size([3, 5]), target size=torch.Size([3, 5])


In [168]:
input.grad
# If x is a Tensor that has x.requires_grad=True then x.grad is another Tensor holding the gradient of x 
# with respect to some scalar value.
# No value actually !

In [169]:
mae_loss = nn.L1Loss() # Define a loss function : The Mean Absolute Error (MAE)
loss = mae_loss(input, target) # Apply the loss function to input and target
print('loss value : ', loss)

loss value :  tensor(1.1193, device='cuda:0', grad_fn=<L1LossBackward>)


loss.backward() computes dloss/dx for every parameter x which has requires_grad=True. 
These are accumulated into x.grad for every parameter x. In pseudo-code: x.grad += dloss/dx

optimizer.step updates the value of x using the gradient x.grad. 
For example, the SGD optimizer performs:
(lr = learning rate)

x += -lr * x.grad

optimizer.zero_grad() clears x.grad for every parameter x in the optimizer. 

It’s important to call this before loss.backward(), otherwise you’ll accumulate the gradients from multiple passes.

If you have multiple losses (loss1, loss2) you can sum them and then call backwards once:

loss3 = loss1 + loss2
loss3.backward()

In [170]:
loss.backward() # compute the gradient of input tensor (the only one with requires_grad=True)

In [171]:
# Get gradient of tensor input saved in a field named grad.
input.grad

tensor([[-0.0667,  0.0667,  0.0667,  0.0667, -0.0667],
        [-0.0667, -0.0667,  0.0667, -0.0667, -0.0667],
        [-0.0667, -0.0667, -0.0667,  0.0667,  0.0667]], device='cuda:0')

In [172]:
input.grad.zero_() # Manually zero the input gradient

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]], device='cuda:0')

In [173]:
input.grad

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]], device='cuda:0')

In [28]:
# Just to verify torch matmul properties
# see : The difference between torch.mul, torch.mm, torch.bmm, torch.matmul
# https://www.programmersought.com/article/80074664295/

import torch

x = torch.randn(2, 1) # torch.Size([2, 1])
A = torch.randn(1, 1) # torch.Size([1, 1])
b = torch.randn(1) # torch.Size([1])

# torch matmul -> If both arguments are 2-dimensional, the matrix-matrix product is returned.
result = torch.matmul(x,A) # torch.Size([2, 1])

print("x={}, A={}".format(x,A))
print("torch.matmul(x,A)={}, nb elem={}".format(result,result.numel()))
print("b={}".format(b))
result2 = result + b # torch.Size([2, 1])
print("torch.matmul(x,A)+b={}".format(result2))

x=tensor([[-0.7048],
        [ 0.2069]]), A=tensor([[-1.7927]])
torch.matmul(x,A)=tensor([[ 1.2635],
        [-0.3710]]), nb elem=2
b=tensor([-0.1028])
torch.matmul(x,A)+b=tensor([[ 1.1607],
        [-0.4738]])


In [43]:
# Just to understand the batched matrix multiply of torch.matmul


In [32]:
# A minimal example of SGD (Stochastic Gradient Descent)
# without a Neural Network

import torch
import torch.nn as nn
import torch.optim as optim

N = 64 # amount of values inside the tensors

# Define 2 equivalent tensors x and y with a torch.Size([64, 1])
x0 = torch.randn(N, 1, requires_grad=False)
x = x0
y = x0

# Define the model parameters to optimize
A = torch.randn(1, 1, requires_grad=True) # size 1 x 1 (weights of the model) -> torch.Size([1, 1])
b = torch.randn(1, requires_grad=True) # Size 1 (biases of the model) -> torch.Size([1])

print("Initial values of A={}, b={}".format(A,b))

optimizer = optim.SGD([A, b], lr=1e-1) # Define SGD with [A, b] as parameters and learning rate to 1e-1

print("Learning rate={}".format(1e-1))

for t in range(20):
    print('-' * 50)
    optimizer.zero_grad() # Clean gradients of A and b (because they have requires_grad to True)
    # torch matmul -> If both arguments are 2-dimensional, the matrix-matrix product is returned.
    y_pred = torch.matmul(x, A) + b # Calculate a noisy predicted value -> (x * A) + b
    loss = ((y_pred - y) ** 2).mean() # Calculate loss value (y is the actual value)
    print("{} -> loss={}".format(t, loss.item())) # print loss value, need item() function to get it
    loss.backward() # Compute gradient of A and b with loss value
    print("A.grad={},b.grad={}".format(A.grad,b.grad))
    optimizer.step() # Update values of A and b by taking account of their gradient and the learning rate
    # A += -lr * A.grad  ; b += -lr * b.grad
    print("A={}, b={}".format(A,b))


Initial values of A=tensor([[0.1336]], requires_grad=True), b=tensor([-0.5767], requires_grad=True)
learning rate=0.1
--------------------------------------------------
0 -> loss=0.7621994018554688
A.grad=tensor([[-1.1614]]),b.grad=tensor([-0.8985])
A=tensor([[0.2497]], requires_grad=True), b=tensor([-0.4869], requires_grad=True)
--------------------------------------------------
1 -> loss=0.5619502663612366
A.grad=tensor([[-1.0094]]),b.grad=tensor([-0.7530])
A=tensor([[0.3507]], requires_grad=True), b=tensor([-0.4116], requires_grad=True)
--------------------------------------------------
2 -> loss=0.41462552547454834
A.grad=tensor([[-0.8765]]),b.grad=tensor([-0.6321])
A=tensor([[0.4383]], requires_grad=True), b=tensor([-0.3483], requires_grad=True)
--------------------------------------------------
3 -> loss=0.3061191439628601
A.grad=tensor([[-0.7604]]),b.grad=tensor([-0.5315])
A=tensor([[0.5144]], requires_grad=True), b=tensor([-0.2952], requires_grad=True)
-------------------------

In [33]:
# A simple test of gradient
# To proof the result of gradient

x = torch.ones(2, 2, requires_grad=True) # size 2 x 2 with all values equal to 1
y = x + 2  # size 2 x 2 with all values equal to 3
z = y * y * 2 # size 2 x 2 with all values equal to 18

out = z.mean() # mean is equal to 18 ! (18*4/4)
print("mean={}".format(out))

out.backward() # compute gradient of x

# Math demonstration :
# out = 1/4 sum of z(j) (for j from 1 to 4)
# d.out/d.x(i) = 1/4 sum(j)[d.z(j)/d.x(i)]  ==> partial derivative of dout with respect to x (i and j from 1 to 4)
# = 1/4 sum(j)[d.2*y(j)*y(j)/d.x(i)]
# = 1/4 sum(j)[ 4*y(j)*(d.y(j)/d.x(i))]
# = 1/4 sum(j)[4*(x(j)+2)*(d.(x(j)+2)/d.x(i))]
# = sum(j)[(x(j)+2)*(d.(x(j)+2)/d.x(i))]
# = x(i)+2    -> d.x(j)/d.x(i) = 0 if i not equal to j
# = 3         -> for x(i) = 1  => tensor x : size 2 x 2 with all 1 values

print("x grad={}".format(x.grad))

mean=18.0
x grad=tensor([[3., 3.],
        [3., 3.]])


In [45]:
# A minimal example of SGD (Stochastic Gradient Descent)
# with a Neural Network

import torch
import torch.nn as nn
import torch.optim as optim
#from torchsummary import summary  # pip install torchsummary

# Let's make some data for a linear regression.
A = 3.1415926
B = 2.7189351
error = 0.1
N = 100 # number of data points

# Data
input_data = torch.randn(N, 1)

# (noisy) Target values that we want to learn.
target = A * input_data + B + (torch.randn(N, 1) * error)

# Creating a model, making the optimizer, defining loss
model = nn.Linear(1, 1) 
# create a Neural Network Linear with in_features=1 and out_features=1 (nn.Linear(in_features,ou_features))
# Applies a linear transformation to the incoming x data: y = x*W^T + b
# Note that the weights W have shape (out_features, in_features) and biases b have shape (out_features).

# Verify model tensors parameters :
for name, param in model.named_parameters():
    print('name: ', name)
    print(type(param))
    print('param.shape: ', param.shape)
    print('param.requires_grad: ', param.requires_grad)
    print('=====')

# name:  weight
# <class 'torch.nn.parameter.Parameter'>
# param.shape:  torch.Size([1, 1])
# param.requires_grad:  True
# =====
# name:  bias
# <class 'torch.nn.parameter.Parameter'>
# param.shape:  torch.Size([1])
#param.requires_grad:  True
# =====

print("model weight={}, model bias={}".format(model.weight,model.bias))

#model_summary(model)

optimizer = optim.SGD(model.parameters(), lr=0.05)
loss_fn = nn.MSELoss()  # the mean squared error (squared L2 norm)

# Run training
nb_iter = 50
for _ in range(0, nb_iter):
    optimizer.zero_grad() # Clear gradient values of model parameters
    predictions = model(input_data)
    # forward input_data into the model
    # So multiply input_data to weight tensor parameter inside model
    # Sizes are : input_data -> torch.Size([100, 1]) ; weight -> torch.Size([1, 1])
    # torch.matmul(input_data,weight) -> torch.Size([100, 1])
    # If both arguments are at least 1-dimensional and at least one argument is N-dimensional (where N > 2), 
    # then a batched matrix multiply is returned
    # Then add bias tensor of size : torch.Size([1])
    # and get a prediction -> torch.Size([100, 1])
    loss = loss_fn(predictions, target) # calculate loss value
    loss.backward() # calculate gradient values
    optimizer.step() # update model parameters with gradient values

    print("-" * 50)
    print("error = {}".format(loss.item()))
    print("learned weight = {}".format(list(model.parameters())[0].data[0, 0]))
    print("learned bias = {}".format(list(model.parameters())[1].data[0]))
    

name:  weight
<class 'torch.nn.parameter.Parameter'>
param.shape:  torch.Size([1, 1])
param.requires_grad:  True
=====
name:  bias
<class 'torch.nn.parameter.Parameter'>
param.shape:  torch.Size([1])
param.requires_grad:  True
=====
model weight=Parameter containing:
tensor([[0.2868]], requires_grad=True), model bias=Parameter containing:
tensor([-0.1752], requires_grad=True)
torch.Size([100, 1])
--------------------------------------------------
error = 19.370567321777344
learned weight = 0.6372024416923523
learned bias = 0.14847417175769806
torch.Size([100, 1])
--------------------------------------------------
error = 15.087311744689941
learned weight = 0.9448572397232056
learned bias = 0.43565675616264343
torch.Size([100, 1])
--------------------------------------------------
error = 11.753310203552246
learned weight = 1.214950442314148
learned bias = 0.6904747486114502
torch.Size([100, 1])
--------------------------------------------------
error = 9.1579008102417
learned weight = 

In [53]:
# An other test with nn.Linear, to verify sizes

input_size = 8
output_size = 14
batch_size = 64

input = torch.FloatTensor(batch_size, input_size) # torch.Size([64, 8])

net = nn.Linear(input_size, output_size) # in_features=8, out_features=14
# net parameters -> W (weight) shape is (out_features, in_features) and b (biases) shape is (out_features)
# W=(14x8), W^T=(8x14), b=(14), input(64x8)
# output = input*W^T + b
# output=(64x14)
# rmq: operator + add tensor b to each line of tensor input*W^T

for name, param in net.named_parameters():
    print('name: ', name)
    print(type(param))
    print('param.shape: ', param.shape)
    print('param.requires_grad: ', param.requires_grad)
    print('=====')

# name:  weight
# <class 'torch.nn.parameter.Parameter'>
# param.shape:  torch.Size([14, 8])
# param.requires_grad:  True
# =====
# name:  bias
# <class 'torch.nn.parameter.Parameter'>
# param.shape:  torch.Size([14])
# param.requires_grad:  True

output = net(input)  # forward input into net

print("Output size:", output.size()) # torch.Size([64, 14])

name:  weight
<class 'torch.nn.parameter.Parameter'>
param.shape:  torch.Size([14, 8])
param.requires_grad:  True
=====
name:  bias
<class 'torch.nn.parameter.Parameter'>
param.shape:  torch.Size([14])
param.requires_grad:  True
=====
Output size: torch.Size([64, 14])


In [62]:
tensor1 = torch.randn(4, 5)
tensor2 = torch.randn(5)
print(tensor1)
print(tensor2)
r = tensor1 + tensor2
print(r)

tensor([[-0.0763,  0.4829, -1.0282, -0.2034, -1.3962],
        [-1.1971, -0.6191, -0.5457,  2.2137, -0.3141],
        [ 0.3867,  0.3600,  0.9878, -1.2267,  1.1280],
        [-0.7363,  0.9165, -0.7225, -0.3447, -1.3103]])
tensor([-1.3968,  0.4827, -0.7991,  0.3998,  0.3016])
tensor([[-1.4731,  0.9656, -1.8273,  0.1964, -1.0946],
        [-2.5939, -0.1364, -1.3448,  2.6135, -0.0125],
        [-1.0101,  0.8427,  0.1887, -0.8269,  1.4295],
        [-2.1331,  1.3993, -1.5217,  0.0551, -1.0087]])
